# 文本向量化

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3',  
                       use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

sentences_1 = ["What is BGE M3?", "Defination of BM25"]
sentences_2 = ["BGE M3 is an embedding model supporting dense retrieval, lexical matching and multi-vector interaction.", 
               "BM25 is a bag-of-words retrieval function that ranks a set of documents based on the query terms appearing in each document"]

embeddings_1 = model.encode(sentences_1, 
                            batch_size=12, 
                            max_length=8192, # If you don't need such a long length, you can set a smaller value to speed up the encoding process.
                            )['dense_vecs']
embeddings_2 = model.encode(sentences_2)['dense_vecs']
similarity = embeddings_1 @ embeddings_2.T
print(similarity)

# 评估脚本

In [14]:
import yaml
import pandas as pd

from tqdm import tqdm
from modules.agent import CommentAnalysisAgent



def read_csv(testset_path, version='default'):
    # 读取测试集
    df = pd.read_csv(testset_path, sep='\t')
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    df['v1_4o_mini'] = df['version']
    df[version] = ''
    print(df.columns)
    return df


def evaluate(robot, testset, testset_path, version):

    length = len(testset)
    for i in tqdm(range(length)):
        comment = testset.iloc[i]['内容']
        if not isinstance(comment, str):
            continue

        response = robot.comment_analyze(comment)
        testset.loc[i, version] = response
    
    testset.to_csv(testset_path, index=False, sep='\t')

    return testset


def metric_calculate(testset, version):
    total_extraction = 0
    correct_extraction = 0
    wrong_extraction = 0
    expected_extraction = 0

    length = len(testset)
    for i in range(length):
        comment = testset.iloc[i]['内容']
        if not isinstance(comment, str):
            continue

        good_points = []
        bad_points = []
        # 很多column是好评点x, 差评点x的格式，从1开始，动态遍历所有这些columns，存储到good_points和bad_points中
        for j in range(1, 100):
            try:
                good_point = testset.iloc[i]['好评点' + str(j)]
            except:
                break
            if pd.notna(good_point):
                good_points.append(good_point)
        for j in range(1, 100):
            try:
                bad_point = testset.iloc[i]['差评点' + str(j)]
            except:
                break
            if pd.notna(bad_point):
                bad_points.append(bad_point)
        
        expected_extraction += len(good_points) + len(bad_points)
        response = testset.iloc[i][version]
        try:
            output = eval(response)
            # 遍历 output，将数据存入 DataFrame
            
            for key, value in output.items():
                total_extraction += 1
                if key[:3] == '好评点':
                    if value in good_points:
                        correct_extraction += 1
                    else:
                        wrong_extraction += 1
                elif key[:3] == '差评点':
                    if value in bad_points:
                        correct_extraction += 1
                    else:
                        wrong_extraction += 1
        except:
            pass
    print('total_extraction:', total_extraction)
    print('correct_extraction:', correct_extraction)
    print('wrong_extraction:', wrong_extraction)
    precision = correct_extraction / total_extraction
    recall = correct_extraction / expected_extraction
    print('precision:', precision)
    print('recall:', recall)

    return precision, recall
        

with open("openai_keys.yaml", "r", encoding="utf-8") as file:
    data = yaml.safe_load(file)

my_key = data['tom']['key']
robot = CommentAnalysisAgent(openai_key=my_key, model="gpt-4o-2024-08-06")  # gpt-4o-2024-08-06  gpt-4o-mini-2024-07-18

version = 'v1_4o'
testset_path = './docs/Comment_Analysis_Testset.csv'
testset = read_csv(testset_path, version)

testset = evaluate(robot, testset, testset_path, version)

print('version: ', version)
precision, recall = metric_calculate(testset, version)

Index(['标题', '内容', '星级', '赞同数', '评论链接', '评论时间', '好评点1', '好评点2', '好评点3', '好评点4',
       '好评点5', '好评点6', '差评点1', '差评点2', '差评点3', '差评点4', '差评点5', '差评点6', '差评点7',
       'version', 'v1', 'v1_4o_mini', 'v1_4o'],
      dtype='object')


100%|██████████| 102/102 [02:58<00:00,  1.75s/it]

version:  v1_4o
total_extraction: 270
correct_extraction: 128
wrong_extraction: 142
precision: 0.4740740740740741
recall: 0.5


In [10]:
print('version': version)
precision, recall = metric_calculate(testset, version)

total_extraction: 309
correct_extraction: 148
wrong_extraction: 161
precision: 0.47896440129449835
recall: 0.578125


# Prompt 测试

In [45]:
import yaml
from openai import OpenAI



class CommentAnalysisAgent:
    def __init__(self, openai_key, model="gpt-3.5-turbo"):
        self.client = OpenAI(api_key=openai_key)
        self.model = model

        self.comment_analyze_prompt = self.get_comment_analyze_prompt()

    def preprocess(self, text):
        text = text.replace('\n', ' ')
        text = text.replace('"', "")
        return text
    
    def comment_analyze(self, comment):
        comment = self.preprocess(comment)
        task_text = self.comment_analyze_prompt + "'" + comment + "'; output=" 
        response = self.one_shot(task_text)
        return response

    def get_comment_analyze_prompt(self):
        prompt = '''You are a review analyst. Your task is to extract aspects from customer reviews, which can be positive, negative or neutral. Follow the order in which the points are mentioned in the review and output whether each point is a positive, negative or neutral aspect along with the review angle. The output format should be:{'Pos1': 'xxx', ..., 'PosN': 'xxx', 'Neg1': 'xxx', ..., 'NegN': 'xxx', 'Neu1': 'xxx', 'NeuN': 'xxx'}, If there are no relevant points, output `{}`.
- The angles for extracting positive and negative review points include: <Installation Difficulty>, <Manual Clarity>, <Installation Time>, <Ease of Moving>, <Packaging Protection>, <Drawer Slide Smoothness>, <Drawer Size>, <Overall Storage Space>, <Wrong or Misfit Parts>, <Extra Power/USB Sockets>, <Cabinet Door Smoothness>, <Customer Service Quality>, <Screw Hole Alignment>, <Odor>, <Missing Parts>, <Noise During Use>, <Legroom Space>, <Hardware Damage>, <Shipping Speed>, <Table Surface Damage>, <Tabletop Material>, <Tabletop Thickness>, <Water Resistance>, <Anti-Slip>, <Stability>, <Aesthetics>, <Edge Finishing>, <Number of Assembly Parts>, <Surface Material>, <Ease of Cleaning>, <Stain Resistance>, <Screw Appearance>, <Cable Management>, <Metal Part Deformation>, <Panel Alignment>, <Return Shipping Cost>, <Shipment Condition>
- Statement like 'the desk is nice/good', 'Good quality' should not be considered as any aspect. Only consider the specific points mentioned in the review.
- A review should have a clear emotional bias to be considered positive or negative.
Example1: comment="Wonderful desk! Wonderful people to work with. Thank you"; output={}
Example2: comment="It's a great looking product once it is all put together. There are a lot of screws, so be careful when screwing the frame to the desk top panels to prevent screws penetrating through the panels. Double check the screw sizes."; output={'Pos1': 'Aesthetics', 'Neu1': 'Number of Assembly Parts'}
Example3: comment="The desk is beautiful looking although it is a little bit deceiving. Kinda wish I ordered a different one. I agree with other reviews that the set up takes forever and there are SO MANY PARTS! I am petite/5'3 and bought a computer chair that is equivalent to the height of a dining room table chair, nothing special and I have a hard time putting my legs under so I do wish it was taller. Also, the drawers aren't as deep as the desk is so that's a bummer they are so small. It's just big enough to fit a 8 x 10 sheet of paper. Overall looks beautiful but functionality can be improved."; output={'Pos1': 'Aesthetics', 'Neg1': 'Installation Time', 'Neg2': 'Number of Assembly Parts', 'Neg3': 'Legroom Space', 'Neg4': 'Drawer Size'}
!Extract only the points explicitly mentioned in the comments, without making any assumptions or inferences. Show the reasoning process without break line and then output the extracted aspects.
Now do: comment='''
        return prompt
    
    def one_shot(self, text):
        completion = self.client.chat.completions.create(
            model=self.model,
            store=True,
            temperature=0.175,
            messages=[
                {"role": "user", "content": text}
            ]
        )
        return completion.choices[0].message.content


with open('./openai_keys.yaml', "r", encoding="utf-8") as file:
    data = yaml.safe_load(file)

my_key = data['openai']['key']
robot = CommentAnalysisAgent(openai_key=my_key, model="gpt-4o-2024-08-06")  # gpt-4o-2024-08-06  gpt-4o-mini-2024-07-18  gpt-4.5-preview-2025-02-27  gpt-4-turbo-2024-04-09

In [46]:
import re
from tqdm import tqdm

comments1 = ["this desk is really nice, however, assembly wise it takes a bit of time to get it properly mounted. I'd suggest they could have made one single panel to facilitate the assembly process and also would give more stability to the desk overall.",
            "It''s a great looking product once it is all put together.  There are a lot of screws , so be careful when screwing the frame to the desk top panels to prevent screws penetrating through the panels. Double check the screw sizes.",
            "Had an issue and customer service helped. Desk took some time to put together but to be expected with any desk.  Wish instructions had a little more detail. But overall not hard just takes a little time.  Desk looks great.  Good quality.",
            "Estoy ensamblando y si aprietas un poquito los tornillos la foto refleja lo que sucede. Pésima Calidad",
            "The desk was very poor quality with defective parts that need to be replaced before the assembly could be finished. I was told these parts were unavailable here and would need to be sent internationally. It would take a minimum of 15 days. So my desk sits here In pieces waiting for the parts. Do you self a favor and proceed with caution if buying this product."]
comments2 = ['''"We purchased the black and white desk. It turned out great, it was a nice upgrade for us.
Things to know:
It is shipped in 2 packages. Our 2nd package was delayed, but customer service was helpful with a tracking number.
The top surface shows every fingerprint, a little annoying, but not terrible. The desk top is like a wood laminate but the rest is good wood.
The drawers are not near as deep as the desk: a little over 13"" compared to 22"" on the desk top.
It is a big project to put together, it took 3 nights for us to complete it."''',
'''"I just purchased and assembled this desk to replace a worn-out desk of similar quality. Here's some notes for those who are considering the same:

* Shipping was fast. The desk shipped within 24 hours from CA. Since I'm in AZ it arrived only a few days later. The desk was packed in 2 boxes, which arrived one day apart. Not surprisingly the boxes are big and heavy, about 65lb and 85lb each. You'll definitely need a dolly or a couple strong people to help move them from your doorstep. The boxes themselves were damaged in transit, but the liberal use of styrofoam (yuck) prevented any damage inside.
* The instructions consisted of diagrams with minimal words. However the steps were mostly understandable and words were used to point out especially critical steps when the diagram wasn't clear enough.  Some steps were still vague however, requiring extra scrutiny or trial-and-error. All parts were clearly labeled and several spares of each screw and nut were generously included.
* The 45-steps took several hours to complete as not a single piece was pre-assembled. Having a helper insert dowels, screws or assemble drawers while you work on the major steps will save a lot of time. Also an electric screwdriver with a phillips and hex head will save a LOT of effort! Thankfully no other tools are required.
* Be sure to assemble the desk in the same room where it will be placed. The 2 sides are not designed to separate, and once assembled it is semi-permanent. Also be aware the cabinet on the left and the drawers on the right are NOT reversible, the pre-drilled holes do not allow for any option besides what is shown.
* For some inexplicable reason, several locknuts and screws are located on the outside of the desk so they are clearly visible. They could have easily been placed on the other side which would have hidden them under the desktop, but the manufacturer must assume this desk will be placed in a corner with both outer sides against the wall. At least small matching stickers are included to hide the some (but not all) of the holes, but it still looks unattractive if any outer side is visible to the room.
* For pre-fab, engineered wood furniture this desk is pretty sturdy. The wood plates are thick and heavy, and the joined pieces are highly reinforced. For example there over 28 locknuts and screws securing the desktop to the body.  The drawers are solid and slide easily and appear they could hold a lot of weight.
* The cabinet does not have any holes or perforations on the back panel. If you plan on placing a PC or other electric equipment inside, you'll have to cut a wiring hole yourself.
* The black surface of the desk looks nice, especially compared to the faux-woodgrain design of other cheap furniture, but it seems every speck of dust and fingerprint is obvious. It's likely the surface will never look pristine!

Overall I'm happy with my purchase. There are some flaws and room for improvement, but I feel like I got my money worth. This isn't something you'll pass on to your grandkids, but its certainly good enough to write this review on!"''',
'''"I absolutely love my new desk. There is room for everything. I upgraded to a corner desk because my new project requires that I use their company laptop, and I still needed to use my company’s laptop. With this desk the layout is perfect.
It took my husband several hours to assemble. He did it a little bit at a time after work for a few nights, so I can’t give an exact time, but it was from 4-6 hours. He’s very good with building and woodworking. He did tell me that someone less skilled at such things would be able to assemble it. The directions were clear.
I have two negatives.
1. One of the corners had about a quarter size chip. I am not bothered by it, but some others may be.
2. The edges are rough. If the edges were beveled, it would be so much better. I’ll need to find a solution as it is irritating to my forearms as I type. I’m sure that I’ll find a solution. They are beveled on the bottom, so it is possible that the manufactured put the drill and bit holes on the wrong side of the boards.
This is a sturdy desk that should last me for many years. It is a huge win for me!"''']

comments3 = ["This is a nice solid desk. Sturdy ,well made. But, a bear to set up.",
             '''"the product image is much wider than the actual product. the desk is only 55” wide, so just measure your space before ordering.

desk looks nice from 5 ft away, but i dont like using it. feels cramped and low quality.

also, it takes forever to put together, and some parts arrived slightly damaged.

overall fine for the price. extra star taken off due to misleading image."''',
'''Delivery was prompt and it had easy to follow directions. Everything was great.''',
'''"I really like this set. It’s super easy to put together and it’s just the right thing for a small at home office or homework desk. I think you can order these two pieces separately too so if you needed multiple homework stations for the kids you could add on a desk. The quality was better than I expected as far as surfaces. I mean it’s laminate but it’s not the cheapest of stuff. Drawer slides are already installed too. Not that that’s a really big deal but it’s one less thing.
If I had the need I would buy this again.
My only issue was the time it took to get here.  Fed Ex is becoming increasingly unreliable, it showed up on Monday but should have arrived the previous Friday. It was still within the original date range but the tracking said Friday from the beginning so I checked it that morning after making arrangements to be home and all the sudden it said Monday. I cant fault the seller for that. If you don’t mind waiting a bit and it’s not a problem, it is worth it though.  Both items did show up together."''',
'''"Let me start by saying, I love this desk and the completed project. It's perfect for my space and what I needed to upgrade.

With that being said, the assembly leaves a lot to be desired. There are NO words on the booklet. It took me almost 30 minutes to figure out how to disassemble the tracks for the drawers to get them on their respective parts.  Personally, these should ship on the drawers/walls already to make it easier.  The steps in the booklet are completely out of order. Do yourself a favor and skip ahead and assemble each drawer, etc individually to make sure you're properly aligned before you get ahead and realize you flubbed step 2 and need to go back.  Don't do the way they do it.  But also be mindful you may need to adjust the tracks again. My top drawer was crooked so I had to remove it, adjust the track down and fit it in.  It can be tricky to get all the parts to align.  That said, this mom did the whole thing alone. The cabinet took longer than the desk, for sure. My one drawer is slightly off but it closes and you can't tell unless you're looking for it.  Its pretty good quality for the money.  Very sleek design. Shipping was great-does come in 2 boxes that may not arrive together!

Would recommend but just be prepared to put in some elbow grease. And the tracks pull apart - if that saves anyone time and mental head slapping 🤣🤣"''',
'''I purchased this desk for my home office. It is the perfect height, looks great in the space, adequate storage for my needs, and very comfortable when I am working at my desk.''',
'''I love everything about this desk and for this price I’m impressed!!! I actually have nothing negative to say about it. Great job! ❤️ it!!''',
'''"Looked nice after I finished. 586 total pieces. Was delivered with one of the drawers bottoms broke.
Looked like a fork truck fork was run into the end of box.  There must be 250 plus holes you need to pilot drill. Don't go to deep you will leave a mark on other side. Test your drill size in some scrap wood. This is not wood. Some kind of processed material. like very dence gray cardboard / drywall. You can't use power driver to install screws, Strips them out. Must do buy hand. I'm pretty handy with mechanical stuff.  Had over 25 hours in unpacking assembly and cleanup. Thats without replace / fix the drawer bottom.
 Good Luck"''',
'''I fell in love with my desk but unfortunately it came damaged. I can’t request a refund according to the description and have to wait a few days for an answer. I wish it was not the front portion itself but it’s visible.''',
'''I love the size of the desk. The problem with the desk is that pieces were missing from the desk. After I assembled the desk by myself, the side with the 3 drawers was clearly the weaker side, and ended up messing up the back piece once I moved it into my room. The drawers seem to get loose from time to time and I know I put them in correctly. I am a person that puts together furniture on the regular and the instructions were a little convoluted in my opinion. I know once I move out of my mom's house, I'll probably throw it away because it won't survive. I'll be on the lookout for a new desk just as large that will be more secure. I would give it a 55%.''',
'''I just received this desk. The box was badly damaged and most of the pieces inside are chipped and even some the wood has split. Don't buy this desk.''',
'''"We were initially unsure on the color based on stock photos but the desk is definitely black. We replaced out the black hardware with gold hardware.

Did take +4hrs to put together but very pleased with the desk!

Highly Recommend!"''',
'''"Great experience and great desk. The desk arrives in two very large rectangle boxes. Very heavy, you will need help bringing it inside. Packed very well, no damage.

The assembly was obviously a bit lengthy given the size of this desk, but there is a YouTube video and paper instructions so it was easy and straightforward. I never got to a point in the instructions where I was confused. You might need the help of a second person on some parts, but I managed to do it all by myself. All the holes align very well. I had no issues along the build.

I have my desk against a wall, and I absolutely love it. There is so much space on this desk, you have room for a computer + so much more. For reference in my pictures, my monitor is 27 inches wide and it fits with so much extra space. Its great.

The desk is strong and sturdy, and is of pretty good quality. If I had to buy a desk over again, it would be this. I 100% made the right choice.

In one of my pictures, you can also see one of their shelves next to the desk as well."''',
'''Looks nice, easy to assemble. It's a little clunky, but overall for what it costs you're not going to find a better desk.''',
'''Sturdy and looks great, definitely recommend''',
'''Just want to write this review as this is an appreciation for customer service from Tribesigns. Product arrived on time and I was assembling it myself. While screwing it lower portion one wooden piece damaged. As in the user manual itself they mentioned to contact the customer care before raising a return request. I contacted thru email and I received reply same day asking for photos and additional details. When I provided they apologized and gave option to replace the full piece. I was looking for replacement part as I already assembled remaining things. As spare parts not available they offered me 50% refund which I accepted and immediately send it Amazon account. Appreciate the customer service and I would love to buy my next product confidently from amazon.''',
'''We love this desk-it is very spacious and we like the black top! Assembly does take a while though.''',
'''This desk is absolutely beautiful! It’s study and with nice size drawers and cabinets. The metal handles are nice touch. The instructions were very simple. If you looking for an elegant desk for your office this is a must have!''',
'''We are missing 4 screws and cannot put the desk together.''',
'''"I purchased the 70in desk and I probably should have payed attention to the measurements because it is very wide lol.
I was able to build the desk on my own. It's pretty heavy but it's very sturdy!"'''
]

def extract_output(text):
    # 正则表达式匹配花括号内的内容，包括花括号本身
    pattern = r'\{.*?\}'
    # 使用 re.findall 获取所有匹配的内容
    matches = re.findall(pattern, text)
    return matches


reps = []
answer = ""
cnt = 0
for comment in tqdm(comments3):
    response = robot.comment_analyze(comment)
    answer += f'\n\n------------\n#{cnt}\n'
    answer += 'comment = ' + comment
    reps.append(response)
    output = extract_output(response)
    answer += '\n\nresponse = ' + response
    cnt += 1

with open('./results/v2_4o.txt', 'w', encoding='utf-8') as file:
    file.write(answer)

In [41]:
print('\n---------\n'.join(reps))

The comment mentions that the desk is nice, but this is not considered an aspect based on the guidelines. The first aspect mentioned is the assembly time, which is described as taking a bit of time, indicating a negative aspect regarding <Installation Time>. The suggestion for a single panel to facilitate the assembly process implies a negative aspect related to <Number of Assembly Parts>. Additionally, the comment suggests that a single panel would provide more stability, indicating a negative aspect regarding <Stability>. Therefore, the extracted aspects are:

{'Neg1': 'Installation Time', 'Neg2': 'Number of Assembly Parts', 'Neg3': 'Stability'}
---------
The review mentions that the product is "great looking," which corresponds to the aspect of <Aesthetics> and is positive. It also mentions "a lot of screws," which corresponds to the aspect of <Number of Assembly Parts> and is neutral because it doesn't express a clear positive or negative sentiment. The rest of the review advises c

In [39]:
print('\n---------\n'.join(reps))

The review mentions several aspects related to the desk:

1. **Shipping Speed**: The second package was delayed. This is a negative aspect because delays are generally considered unfavorable.
2. **Customer Service Quality**: Customer service was helpful with providing a tracking number. This is a positive aspect as it indicates good customer service.
3. **Table Surface Material**: The top surface shows every fingerprint, which is a negative aspect because it implies the material is prone to showing marks easily.
4. **Drawer Size**: The drawers are not as deep as the desk, which is a negative aspect because it suggests limited storage space in the drawers.
5. **Installation Time**: It took 3 nights to complete the assembly, indicating a lengthy installation process, which is a negative aspect.

Based on these observations, the extracted aspects are: {'Neg1': 'Shipping Speed', 'Pos1': 'Customer Service Quality', 'Neg2': 'Table Surface Material', 'Neg3': 'Drawer Size', 'Neg4': 'Installati

In [51]:
english_version = '''Installation Difficulty
Manual Clarity
Installation Time
Ease of Moving
Packaging Protection
Drawer Slide Smoothness
Drawer Size
Overall Storage Space
Wrong or Misfit Parts
Extra Power/USB Sockets
Cabinet Door Smoothness
Customer Service Quality
Screw Hole Alignment
Odor
Missing Parts
Noise During Use
Legroom Space
Hardware Damage
Shipping Speed
Table Surface Damage
Tabletop Material
Tabletop Thickness
Water Resistance
Anti-Slip
Stability
Aesthetics
Edge Finishing
Number of Assembly Parts
Surface Material
Ease of Cleaning
Stain Resistance
Screw Appearance
Cable Management
Metal Part Deformation
Panel Alignment
Return Shipping Cost
Shipment Condition'''.split('\n')
chinese_version = '''安装难度
说明书内容
安装时长
搬运难度
包装保护性
抽屉滑轨顺畅度
抽屉大小
整体收纳空间
错件情况
附加电源USB插座
柜门开合顺畅度
客服质量
螺丝孔位匹配度
气味情况
缺件情况
使用噪音
腿部空间大小
五金件损坏情况
物流速度
桌体破损情况
桌板材质
桌板厚度
桌面防水性
防滑性
稳定性
美观度
桌面边缘处理
需拼装组件数量
桌面材质
易清洁性质
耐脏性
螺丝钉外观
线缆管理
金属零部件变形情况
多板平齐情况
退货运输费用
运输货品完好度'''.split('\n')

# for i in range(len(english_version)):
#     print('"' + english_version[i] + '": "' + chinese_version[i] + '" ,')
print('>, <'.join(chinese_version))

安装难度>, <说明书内容>, <安装时长>, <搬运难度>, <包装保护性>, <抽屉滑轨顺畅度>, <抽屉大小>, <整体收纳空间>, <错件情况>, <附加电源USB插座>, <柜门开合顺畅度>, <客服质量>, <螺丝孔位匹配度>, <气味情况>, <缺件情况>, <使用噪音>, <腿部空间大小>, <五金件损坏情况>, <物流速度>, <桌体破损情况>, <桌板材质>, <桌板厚度>, <桌面防水性>, <防滑性>, <稳定性>, <美观度>, <桌面边缘处理>, <需拼装组件数量>, <桌面材质>, <易清洁性质>, <耐脏性>, <螺丝钉外观>, <线缆管理>, <金属零部件变形情况>, <多板平齐情况>, <退货运输费用>, <运输货品完好度


In [ ]:
'''
1. 不通过。整体大部分分析正确，只有一处，商品图片和货物不一致不是指商品的Aesthetics，但这个是规则里去掉了货物与不一致
2. 通过
3. 通过，但是有一个中性分析是错误的，不过目前考虑好差评优先
4. 通过
5. 通过
6. 通过
7. 不通过。有一处错误，panel alignment那里
8. 通过
9. 通过，但是很多分析比较看个人理解，逻辑不能说错
10. 通过
11. 通过，这里installation time确实是偏负面的因为有but，不过应该情绪不激烈
12. 通过，漏了一个差评点Installation Time
13. 通过
14. 不通过，词提取全对，但应该都是好评，由于太短模型认为是中性了
15. 通过
16. 通过
17. 通过，不过cabinet和drawer都提了很好的size所以认为有overall storage space的好评可能不完全精准
18. 通过
19. 通过
20. 通过，但是漏了一个stability
'''

In [57]:
import re

def extract_output(text):
    # 正则表达式匹配花括号内的内容，包括花括号本身，并使用 re.DOTALL 匹配多行内容
    pattern = r'\{.*?\}'
    # 使用 re.findall 获取所有匹配的内容，并设置 re.DOTALL 标志
    matches = re.findall(pattern, text, re.DOTALL)

    matches = [match.replace('\n', '') for match in matches]
    return matches

In [60]:
800*7/60

93.33333333333333